# Data Extraction

Loads a dataset with product data

In [104]:
import numpy as np
import pandas as pd
import os

database_path  = os.environ['DATASET_PATH']
data = pd.read_csv(filepath_or_buffer=database_path, sep=',', header=0)

# Data Formatting

É importante entender um pouco como estão os dados no dataset para se ter uma ideia de que informação poderia ser utilizado para predizer a categoria de cada produto

In [105]:
print('Tipos de dados do dataset: \n' + str(data.dtypes))
print('\nQuantidade de linhas no dataset: ' + str(data.shape[0]))
print('Quantidade de colunas no dataset: ' + str(data.shape[1]))
print('\n')
print(data.count())
data.head()

Tipos de dados do dataset: 
product_id             int64
seller_id              int64
query                 object
search_page            int64
position               int64
title                 object
concatenated_tags     object
creation_date         object
price                float64
weight               float64
express_delivery       int64
minimum_quantity       int64
view_counts            int64
order_counts         float64
category              object
dtype: object

Quantidade de linhas no dataset: 38000
Quantidade de colunas no dataset: 15


product_id           38000
seller_id            38000
query                38000
search_page          38000
position             38000
title                38000
concatenated_tags    37998
creation_date        38000
price                38000
weight               37942
express_delivery     38000
minimum_quantity     38000
view_counts          38000
order_counts         17895
category             38000
dtype: int64


,product_id,seller_id,query,search_page,position,title,concatenated_tags,creation_date,price,weight,express_delivery,minimum_quantity,view_counts,order_counts,category
0,11394449,8324141,espirito santo,2,6,Mandala Espírito Santo,mandala mdf,2015-11-14 19:42:12,171.890000,1200.0,1,4,244,NaN,Decoração
1,15534262,6939286,cartao de visita,2,0,Cartão de Visita,cartao visita panfletos tag adesivos copos lon...,2018-04-04 20:55:07,77.670000,8.0,1,5,124,NaN,Papel e Cia
2,16153119,9835835,expositor de esmaltes,1,38,Organizador expositor p/ 70 esmaltes,expositor,2018-10-13 20:57:07,73.920006,2709.0,1,1,59,NaN,Outros
3,15877252,8071206,medidas lencol para berco americano,1,6,Jogo de Lençol Berço Estampado,t jogo lencol menino lencol berco,2017-02-27 13:26:03,118.770004,0.0,1,1,180,1.0,Bebê
4,15917108,7200773,adesivo box banheiro,3,38,ADESIVO BOX DE BANHEIRO,adesivo box banheiro,2017-05-09 13:18:38,191.810000,507.0,1,6,34,NaN,Decoração


O "Título" dos produtos pode ser uma boa forma de prever a categoria e, para manter a simplicidade, somente ele será avaliado no treinamento do modelo para predizer a categoria dos produtos

In [106]:
titles = list(data['title'])
categories = list(data['category'])

training_size = int(len(titles) * 0.8)

train_titles = titles[:training_size]
train_target_categories = categories[:training_size]

test_titles = titles[training_size:]
test_target_categories = categories[training_size:]

Como o "Título" é um campo texto composto por mútliplas palavras, é necessário transformar ele numa representação numérica para poder fazer o treinamento do modelo. Isso será feito usando a técnica "Bag of Words"

In [107]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
train_features = vectorizer.fit_transform(train_titles)

# Modeling

Foram feitos testes com os modelos para classificação 'MultinomialNB', 'DecisionTreeClassifier', 'LinearSVC' e 'RandomForestClassifier'. Mas o que apresentou o melhor resultado foi o 'RandomForestClassifier', apesar de demorar um pouco dados os hiperparametros padrão terem sido considerados.

In [108]:
%time
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier()
classifier.fit(train_features.toarray(), train_target_categories)

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 12.9 µs


RandomForestClassifier()

# Model Validation

Para a validação do modelo foi utilizado o método classification_report por já montar um relatório formatado com as métricas gerais e para cada valor de "Categoria" encontrado no dataset

In [109]:
from sklearn.metrics import classification_report
import json

test_features = vectorizer.transform(test_target_categories)
prediction = classifier.predict(test_features)

classification = classification_report(test_categories, prediction, zero_division=True, digits=6)
print(classification)

file = open(os.environ['METRICS_PATH'], 'w')
file.write(format(classification))
file.close()

                    precision    recall  f1-score   support

              Bebê   1.000000  1.000000  1.000000      1415
Bijuterias e Jóias   1.000000  0.000000  0.000000       197
         Decoração   1.000000  1.000000  1.000000      1713
     Lembrancinhas   0.896886  1.000000  0.945640      3514
            Outros   1.000000  0.000000  0.000000       207
       Papel e Cia   1.000000  1.000000  1.000000       554

          accuracy                       0.946842      7600
         macro avg   0.982814  0.666667  0.657607      7600
      weighted avg   0.952323  0.946842  0.921708      7600



# Model Exportation

In [110]:
import pickle

with open(os.environ['MODEL_PATH'], 'wb') as f:
    pickle.dump(classifier, f)